# Data cleaning 
This notebook carries out the complete preprocessing workflow for the retail sales datasets used in the project. The objective is to ensure that all source tables are clean, consistent, and ready for further exploratory analysis, modeling, and dashboarding.

## Initialization (downloading data and inspecting it)

In [1]:
# Import necessary libraries
import pandas as pd
import os
import pyarrow.parquet as pq

In [2]:
# Setting up paths
project_root = os.path.dirname(os.getcwd())
data_path = os.path.join(project_root, "data_raw")

print("Project root:", project_root)
print("Data path:", data_path)
print("Files:", os.listdir(data_path))

# Loading raw data
df_cat = pd.read_csv(os.path.join(data_path, "categorias.csv"))
df_clientes = pd.read_csv(os.path.join(data_path, "clientes.csv"))
df_metodos = pd.read_csv(os.path.join(data_path, "metodos_pago.csv"))
df_prod = pd.read_csv(os.path.join(data_path, "productos.csv"))
df_ventas = pd.read_csv(os.path.join(data_path, "ventas.csv"))

datasets = {
    "Categorias": df_cat,
    "Clientes": df_clientes,
    "Metodos de Pago": df_metodos,
    "Productos": df_prod,
    "Ventas": df_ventas
}

Project root: /Users/sofiaknutas/Desktop/Reto_MA2003b/reto_ma2003b
Data path: /Users/sofiaknutas/Desktop/Reto_MA2003b/reto_ma2003b/data_raw
Files: ['metodos_pago.csv', 'categorias.csv', 'clientes.csv', 'ventas.csv', 'productos.csv']


In [3]:
# Inspecting datasets
for name, df in datasets.items():
    print(f"===== {name} Overview ====================")
    print(df.head(), "\n")
    print(df.info(), "\n")
    print(df.describe(include="all"), "\n")
    print("Missing values:\n", df.isna().sum())

===== Categorias Overview ====================
   ID_Categoria          Categoría  \
0             1            Lácteos   
1             2         Carnicería   
2             3          Panadería   
3             4  Frutas y Verduras   
4             5         Congelados   

                                         Descripción  
0  Productos lácteos frescos y procesados, como l...  
1  Carnes frescas y procesadas, como carne de vac...  
2  Productos horneados frescos, como pan, factura...  
3  Frutas y verduras frescas, locales e importada...  
4  Productos congelados, como papas fritas, empan...   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID_Categoria  8 non-null      int64 
 1   Categoría     8 non-null      object
 2   Descripción   8 non-null      object
dtypes: int64(1), object(2)
memory usage: 324.0+ bytes
None 

        ID_Categoria

Based on the initial inspection, below is a brief summary of each table.

#### Categorías
- 8 rows, 3 columns: `ID_Categoria`, `Categoría`, `Descripción`.
- No missing values.
- Each category appears once and has a clear text description.

#### Clientes
- 326 rows, 6 columns: `ID_Cliente`, `Nombre`, `Apellido`, `Email`, `Fecha_Resgistro`, `Región`.
- No missing values.
- Several regions, with Buenos Aires as the most frequent.
- `Fecha_Resgistro` is stored as text and could benefit from being converted to date.
- No duplicates. 

#### Métodos de Pago
- 5 rows, 3 columns: `ID_Metodo`, `Método`, `Descripción`.
- No missing values.
- One row per payment method, with a short description of each.

#### Productos
- 38 rows, 5 columns: `ID_Producto`, `Nombre_producto`, `Categoría`, `Precio_Unitario`, `Stock`.
- No missing values.
- `Precio_Unitario` is stored as a string with comma as decimal separator, which should be converted to numeric.

#### Ventas
- 3029 rows, 7 columns: `ID_Venta`, `Fecha`, `ID_Cliente`, `ID_Producto`, `Cantidad`, `Método_Pago`, `Estado`.
- No missing values.
- `Fecha` is stored as an object and appears in slightly different formats (e.g. `31/01/2024` and `31/1/2024`), so it needs standardization and conversion to date type.
- All rows have `Estado = "Completa"`, meaning all transactions has gone through.

#### Conclusion

The raw data is complete without missing values, but several preprocessing steps are required before analysis:
- Convert text dates (`Fecha`, `Fecha_Resgistro`) to datetime.
- Convert `Precio_Unitario` to numeric.

## Handling of duplicates

In [4]:
# Check for duplicates
for name, df in datasets.items():
    duplicates = df.duplicated().sum()
    print(f"{name} - Duplicates: {duplicates}")

Categorias - Duplicates: 0
Clientes - Duplicates: 0
Metodos de Pago - Duplicates: 0
Productos - Duplicates: 0
Ventas - Duplicates: 29


In [5]:
# Removing duplicates from ventas dataset
df_ventas = df_ventas.drop_duplicates()

for name, df in datasets.items():
    duplicates = df.duplicated().sum()
    print(f"{name} - Duplicates: {duplicates}")


Categorias - Duplicates: 0
Clientes - Duplicates: 0
Metodos de Pago - Duplicates: 0
Productos - Duplicates: 0
Ventas - Duplicates: 29


 In the Ventas dataFrame 29 duplicate recorords were detected. These repeated entries made the sales numbers look higher than they really were and added noise that could harm both the clustering and prediction models. To fix this, the duplicated records were dropped.

## Handle datatypes and outliers

In [12]:
# Convert price column to numeric
if "Precio_Unitario" in df_prod.columns:
        df_prod["Precio_Unitario"] = (
        df_prod["Precio_Unitario"]
        .astype(str)              
        .str.replace(",", ".", regex=False)
        .astype(float)
    )

# Create the IQR outlier function
def iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower) | (df[column] > upper)]
    return outliers, lower, upper


# Define which columns are suitable for outlier detection in each dataset
columns_to_check = {
    "categorias": [],  # no numeric fields
    "clientes": [],    # ID and region are categorical
    "metodos_pago": [], # only IDs and text
    "productos": ["Precio_Unitario", "Stock"],
    "ventas": ["Cantidad"]
}

datasets = {
    "categorias": df_cat,
    "clientes": df_clientes,
    "metodos_pago": df_metodos,
    "productos": df_prod,
    "ventas": df_ventas,
}

# Run the outlier detection for each dataset

for name, df in datasets.items():
    print(f"\n===== OUTLIER ANALYSIS: {name.upper()} =====")

    if len(columns_to_check[name]) == 0:
        print("Not suitable for outlier analysis.\n")
        continue

    for col in columns_to_check[name]:
        print(f"\n--- Checking column: {col} ---")
        outliers, low, up = iqr(df, col)

        print(f"Lower bound: {low:.3f}, Upper bound: {up:.3f}")
        print(f"Outliers found: {len(outliers)}")

        if len(outliers) > 0:
            print(outliers.head())
        else:
            print("No outliers detected.")


===== OUTLIER ANALYSIS: CATEGORIAS =====
Not suitable for outlier analysis.


===== OUTLIER ANALYSIS: CLIENTES =====
Not suitable for outlier analysis.


===== OUTLIER ANALYSIS: METODOS_PAGO =====
Not suitable for outlier analysis.


===== OUTLIER ANALYSIS: PRODUCTOS =====

--- Checking column: Precio_Unitario ---
Lower bound: -5.215, Upper bound: 23.225
Outliers found: 0
No outliers detected.

--- Checking column: Stock ---
Lower bound: -606.500, Upper bound: 6829.500
Outliers found: 0
No outliers detected.

===== OUTLIER ANALYSIS: VENTAS =====

--- Checking column: Cantidad ---
Lower bound: -2.500, Upper bound: 9.500
Outliers found: 0
No outliers detected.


In [13]:
df_prod = df_prod[df_prod["Precio_Unitario"] != 28.56]

for name, df in datasets.items():
    print(f"\n===== OUTLIER ANALYSIS: {name.upper()} =====")

    if len(columns_to_check[name]) == 0:
        print("Not suitable for outlier analysis.\n")
        continue

    for col in columns_to_check[name]:
        print(f"\n--- Checking column: {col} ---")
        outliers, low, up = iqr(df, col)

        print(f"Lower bound: {low:.3f}, Upper bound: {up:.3f}")
        print(f"Outliers found: {len(outliers)}")

        if len(outliers) > 0:
            print(outliers.head())
        else:
            print("No outliers detected.")


===== OUTLIER ANALYSIS: CATEGORIAS =====
Not suitable for outlier analysis.


===== OUTLIER ANALYSIS: CLIENTES =====
Not suitable for outlier analysis.


===== OUTLIER ANALYSIS: METODOS_PAGO =====
Not suitable for outlier analysis.


===== OUTLIER ANALYSIS: PRODUCTOS =====

--- Checking column: Precio_Unitario ---
Lower bound: -5.215, Upper bound: 23.225
Outliers found: 0
No outliers detected.

--- Checking column: Stock ---
Lower bound: -606.500, Upper bound: 6829.500
Outliers found: 0
No outliers detected.

===== OUTLIER ANALYSIS: VENTAS =====

--- Checking column: Cantidad ---
Lower bound: -2.500, Upper bound: 9.500
Outliers found: 0
No outliers detected.


Categorías, Clientes and Métodos de Pago do not include numerical variables suitable for outlier detection. In Productos, one outlier was found in `Precio_Unitario`, where one product Asado was priced at 28.56, which is above the upper bound of 24.987. This is a realistic high-value product, however to be able to produe more clear clusters, this product was removed.
No outliers appeared in Stock.
In Ventas, the variable `Cantidad` showed no outliers.

In [14]:
# Adjust date columns to datetime format
df_ventas["Fecha"] = pd.to_datetime(df_ventas["Fecha"], dayfirst=True)
df_clientes["Fecha_Resgistro"] = pd.to_datetime(df_clientes["Fecha_Resgistro"], dayfirst=True)

print("Datatype:", df_ventas["Fecha"].dtype)
print("Datatype:", df_clientes["Fecha_Resgistro"].dtype)

Datatype: datetime64[ns]
Datatype: datetime64[ns]


## Creating a final cleaned dataframe and saving

We have now checked and handled missing values, duplicates, outliers and data types. With this, the dataset can be considered clean, and the next step is to create a master table for further analysis. This table will be saved as a parquet file in the directory `data_cleaned/master.parquet`. 

In [17]:
# Merging datasets to create master dataframe
df = (
    df_ventas
    .merge(df_clientes, on="ID_Cliente", how="left")
    .merge(df_prod, on="ID_Producto", how="left")
    .merge(df_cat, on="Categoría", how="left")
    .merge(df_metodos, left_on="Método_Pago", right_on="ID_Metodo", how="left")
)

# Extracting year, month, and week from the date
df["anio"] = df["Fecha"].dt.year
df["mes"] = df["Fecha"].dt.month
df["semana"] = df["Fecha"].dt.isocalendar().week

# Calculating revenue and ticket promedio
df["ingreso"] = df["Cantidad"] * df["Precio_Unitario"]

# Saving the cleaned master dataframe in parquet format
clean_path = os.path.join(project_root, "data_cleaned")
output_file = os.path.join(clean_path, "master.parquet")
df.to_parquet(output_file)
print("File saved to:", output_file)


File saved to: /Users/sofiaknutas/Desktop/Reto_MA2003b/reto_ma2003b/data_cleaned/master.parquet
